In [1]:
from pytket.circuit import Circuit, CircBox, QControlBox

from pytket.circuit.display import render_circuit_jupyter

from pytket.extensions.nexus import NexusBackend, QuantinuumConfig, Nexus
from datetime import datetime

from pytket.backends.backendresult import BackendResult

import phayes

import numpy as np
import matplotlib.pyplot as plt

In [9]:
def prepare_circ(circ: Circuit) -> Circuit():
    #
    n_qubits = circ.n_qubits
    prepped_circ = Circuit(n_qubits)
    
    for i in range(n_qubits):
        prepped_circ.H(i)
    return prepped_circ

def construct_unitary(n_qubits: Int) -> Circuit():
    
    
def bayes_qpe(prepped_circ: Circuit, cu_circ: Circuit, theta) -> Circuit():
    qpe_circ = Circuit()
    prep_n_qubits = prepped_circ.n_qubits
    prepped_reg = qpe_circ.add_q_register("p", prep_n_qubits)

    qpe_circ.add_circuit(cu_circ, list(prepped_reg))    
    qpe_circ.add_circuit(prepped_circ, list(prepped_reg))
    qpe_circ.measure_register(prepped_reg, "c")
    return qpe_circ


test_circ = Circuit(4)
prepped_circ = prepare_circ(test_circ)
test_u = Circuit(4).X(1).X(3)
qpe_circ = bayes_qpe(prepped_circ, test_u, 0)
render_circuit_jupyter(qpe_circ)